<a href="https://colab.research.google.com/github/Dyakonov/python_hacks/blob/master/dj_LSTMLangModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Языковая модель на LSTM

по мотивам https://atcold.github.io/pytorch-Deep-Learning/

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import time

# import shakespeare_data as sh

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

## Загрузить данные

In [ ]:
# filename = 'onegin_small.txt'
filename = 'onegin.txt'

import re



def read_corpus(filename):
    # r = re.compile("[а-яА-Я .!,;:]+")
    lines = []
    with open(filename, 'r', encoding='Windows-1251', errors='ignore') as f:
        for pos, line in enumerate(f):
            # line = line.replace("\t", "").replace("\n", " ")
            #line = ''.join([c for c in filter(r.match, line)]) # оставить русские буквы
            #
            line = re.sub('[^а-яА-Я .!,;:]+', ' ', line.replace("\t", "").replace("\n", " ")).strip().lower()
            line = re.sub(" +", " ", line) # схлопываем пробелы
            line = line.replace(" .", ".")
            line = line.replace(" ,", ",")
            line = line.replace(" !", "!")
            line = re.sub("[.]+", ".", line)
            line = re.sub("[,]+", ",", line)
            line = re.sub("[!]+", "!", line)
            if len(line.strip()) > 0:
                lines.append(line)
    corpus = " ".join(lines)
    return corpus

corpus = read_corpus(filename)
print (corpus[:2000])

александр сергеевич пушкин евгений онегин роман в стихах ,. проникнутый тщеславием, он обладал сверх того еще особенной гордостью, которая побуждает признаваться с одинаковым равнодушием в своих как добрых, так и дурных поступках, следствие чувства превосходства, быть может мнимого.из частного письма фр. мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной, высоких дум и простоты; но так и быть рукой пристрастной прими собранье пестрых глав, полусмешных, полупечальных, простонародных, идеальных, небрежный плод моих забав, бессонниц, легких вдохновений, незрелых и увядших лет, ума холодных наблюдений и сердца горестных замет. глава первая и жить торопится, и чувствовать спешит. князь вяземский эпиграф взят из стихотворения п. а. вяземского первый снег. мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог. его при

In [ ]:
def get_charmap(corpus):
    chars = list(set(corpus))
    chars.sort()
    charmap = {c: i for i, c in enumerate(chars)}
    return chars, charmap


def map_corpus(corpus, charmap):
    return np.array([charmap[c] for c in corpus], dtype=np.int64)


def to_text(line, charset):
    return "".join([charset[c] for c in line])

print(f"Число символов в корпусе: {len(corpus)}")
chars, charmap = get_charmap(corpus)
charcount = len(chars)
print(f"Число уникальных символов: {len(chars)}")
corpus_array = map_corpus(corpus, charmap)
print(f"corpus_array.shape: {corpus_array.shape}")

# Число символов в корпусе: 158663
# Число уникальных символов: 148
# corpus_array.shape: (158663,)

Число символов в корпусе: 147224
Число уникальных символов: 38
corpus_array.shape: (147224,)


In [ ]:
print (' '.join(chars)) # символы

  ! , . : ; а б в г д е ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я


In [ ]:
print (charmap)

{' ': 0, '!': 1, ',': 2, '.': 3, ':': 4, ';': 5, 'а': 6, 'б': 7, 'в': 8, 'г': 9, 'д': 10, 'е': 11, 'ж': 12, 'з': 13, 'и': 14, 'й': 15, 'к': 16, 'л': 17, 'м': 18, 'н': 19, 'о': 20, 'п': 21, 'р': 22, 'с': 23, 'т': 24, 'у': 25, 'ф': 26, 'х': 27, 'ц': 28, 'ч': 29, 'ш': 30, 'щ': 31, 'ъ': 32, 'ы': 33, 'ь': 34, 'э': 35, 'ю': 36, 'я': 37}


In [ ]:
# текст -> последовательности фиксированной длины
# плохая версия!!!

class TextDataset(Dataset):

    def __init__(self, text, seq_len = 200):
        n_seq = len(text) // seq_len
        text = text[:n_seq * seq_len]
        self.data = torch.tensor(text).view(-1,seq_len)

    def __getitem__(self,i):
        txt = self.data[i]
        return txt[:-1], txt[1:] # метки - это те же последовательности, сдвинутые на 1

    def __len__(self):
        return self.data.size(0)

# используется в DataLoader - список последовательностей в батч
# ответ: seq_len x batch_size
def collate(seq_list):
    inputs = torch.cat([s[0].unsqueeze(1) for s in seq_list], dim=1)
    targets = torch.cat([s[1].unsqueeze(1) for s in seq_list], dim=1)
    return inputs, targets

In [ ]:
"""
более хорошая версия
"""
class TextDataset(Dataset):

    def __init__(self, text, seq_len = 200):
        self.len = len(text) - seq_len + 1
        self.data = []
        self.seq_len = seq_len
        for i in range(self.len):
            self.data.append(torch.tensor(text[i: i+self.seq_len]))

    def __getitem__(self, i):
        #line = self.data[i: i+self.seq_len]
        #line = torch.tensor(line) # это плохо
        line = self.data[i]
        return line[:-1].to(DEVICE), line[1:].to(DEVICE)

    def __len__(self):
        return self.len

In [ ]:
# Model
class CharLanguageModel(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, nlayers):
        super(CharLanguageModel,self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.nlayers = nlayers
        self.embedding = nn.Embedding(vocab_size,
                                      embed_size) # Embedding layer
        self.rnn = nn.LSTM(input_size = embed_size,
                           hidden_size=hidden_size,
                           num_layers=nlayers) # Recurrent network
        self.scoring = nn.Linear(hidden_size, vocab_size) # Projection layer

    def forward(self, seq_batch): # L x N
        # returns 3D logits
        batch_size = seq_batch.size(1) # здесь это размерность 1
        embed = self.embedding(seq_batch) # L x N x E
        hidden = None
        output_lstm, hidden = self.rnn(embed, hidden) # L x N x H
        output_lstm_flatten = output_lstm.view(-1, self.hidden_size) # (L*N) x H
        output_flatten = self.scoring(output_lstm_flatten) #(L*N) x V
        return output_flatten.view(-1, batch_size, self.vocab_size)

    def generate(self, seq, n_words): # L x V
        # жадный поиск для генерации слов
        generated_words = []
        embed = self.embedding(seq).unsqueeze(1) # L x 1 x E
        hidden = None
        output_lstm, hidden = self.rnn(embed, hidden) # L x 1 x H
        output = output_lstm[-1] # 1 x H
        scores = self.scoring(output) # 1 x V
        _, current_word = torch.max(scores, dim=1) # 1 x 1
        generated_words.append(current_word)
        if n_words > 1:
            for i in range(n_words-1):
                embed = self.embedding(current_word).unsqueeze(0) # 1 x 1 x E
                output_lstm, hidden = self.rnn(embed, hidden) # 1 x 1 x H
                output = output_lstm[0] # 1 x H
                scores = self.scoring(output) # V
                _,current_word = torch.max(scores, dim=1) # 1
                generated_words.append(current_word)
        return torch.cat(generated_words, dim=0)

In [ ]:
def train_epoch(model, optimizer, train_loader, val_loader):
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(DEVICE)
    train_loss = 0
    before = time.time()
    print("training", len(train_loader), "number of batches")
    for batch_idx, (inputs,targets) in enumerate(train_loader):
        if batch_idx == 0:
            first_time = time.time()
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        outputs = model(inputs) # 3D
        loss = criterion(outputs.view(-1,outputs.size(2)),targets.view(-1)) # Loss of the flattened outputs
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        #if batch_idx == 0:
        #    print("Time elapsed", time.time() - first_time)

        #if batch_idx % 500 == 0 and batch_idx != 0:
        #    after = time.time()
        #    print("Time: ", after - before)
        #    print("Loss per word: ", loss.item() / batch_idx)
        #    print("Perplexity: ", np.exp(loss.item() / batch_idx))
        #    after = before

    train_loss = train_loss / batch_idx

    val_loss = 0
    batch_id = 0
    for inputs,targets in val_loader:
        batch_id += 1
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        outputs = model(inputs)
        loss = criterion(outputs.view(-1,outputs.size(2)), targets.view(-1))
        val_loss += loss.item()
    val_lpw = val_loss / batch_id
    # print("\nValidation loss per word:",val_lpw)
    print("Train perplexity :", np.exp(train_loss))
    print("Validation perplexity :", np.exp(val_loss / batch_id))
    return val_lpw

In [ ]:
#model = CharLanguageModel(charcount, 256, 256,3)
model = CharLanguageModel(vocab_size=charcount,
                          embed_size=256,
                          hidden_size=256,
                          nlayers=2)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.01, weight_decay=1e-7) #, # lr=0.001,
                             #weight_decay=1e-6)

split = 120000
train_dataset = TextDataset(corpus_array[:split], seq_len=100)
val_dataset = TextDataset(corpus_array[split:], seq_len=100)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64, collate_fn = collate)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=64, collate_fn = collate, drop_last=True)

In [ ]:
#train_dataset = TextDataset(shakespeare_array, seq_len = 10)
#train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1, collate_fn = collate)

In [ ]:
train_dataset.__len__(), val_dataset.__len__()
# (1200, 386)

(119901, 27125)

In [ ]:
# i = 0
# for i1, i2 in val_loader:
#     i = i + 1
#     if (i>10): break;
#     # print(i1.shape, i2.shape)
#     print(i1, i2)

128 3 0.01
Train perplexity : 1.708431299024099
Validation perplexity : 68.95001547975727

256 3 lr=0.01, weight_decay=1e-7
Train perplexity : 1.6952816075689339
Validation perplexity : 55.76105459243429

256 2 lr=0.01, weight_decay=1e-7
Train perplexity : 1.9566967733083707
Validation perplexity : 32.95505913167213

In [ ]:
for i in range(2):
    train_epoch(model=model,
                optimizer=optimizer,
                train_loader=train_loader,
                val_loader=val_loader)

training 1874 number of batches
Train perplexity : 3.100737592348649
Validation perplexity : 20.92226048805598
training 1874 number of batches
Train perplexity : 1.68108752638697
Validation perplexity : 44.035060143245296


In [ ]:
def generate(model, seed, nwords):
    seq = map_corpus(seed, charmap)
    seq = torch.tensor(seq).to(DEVICE)
    out = model.generate(seq, nwords)
    return to_text(out.cpu().detach().numpy(), chars)

In [ ]:
print(generate(model, "онегин встал и подошел, сказав, что", 100))

 значит видеть свет! где ж лучше гордым и после важно повторять одно, стараться вас задригалы иль пр


In [ ]:
print(generate(model, "мой дядя самых честных правил, когда не в шутку", 100))

 занеможественной мечты, поэзии живой и кума ему не наши внемя. тут же полуравет, и вдруг нетвал, пр


In [ ]:
seq = map_corpus(corpus[500:530], charmap) # "Высоких дум и простоты"
seq = torch.tensor(seq).to(DEVICE)
out = model.generate(seq, 100)


In [ ]:
corpus[:1000] # 500:1530]

'александр сергеевич пушкин евгений онегин роман в стихах ,. проникнутый тщеславием, он обладал сверх того еще особенной гордостью, которая побуждает признаваться с одинаковым равнодушием в своих как добрых, так и дурных поступках, следствие чувства превосходства, быть может мнимого.из частного письма фр. мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной, высоких дум и простоты; но так и быть рукой пристрастной прими собранье пестрых глав, полусмешных, полупечальных, простонародных, идеальных, небрежный плод моих забав, бессонниц, легких вдохновений, незрелых и увядших лет, ума холодных наблюдений и сердца горестных замет. глава первая и жить торопится, и чувствовать спешит. князь вяземский эпиграф взят из стихотворения п. а. вяземского первый снег. мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог. его пр

In [ ]:
out

tensor([21, 20, 23, 24,  6, 17, 34, 19, 20,  0, 21, 20, 23, 24,  6, 17, 34, 19,
        20,  0, 21, 20, 23, 24,  6, 17, 34, 19, 20,  0, 21, 20, 23, 24,  6, 17,
        34, 19, 20,  0, 21, 20, 23, 24,  6, 17, 34, 19, 20,  0, 21, 20, 23, 24,
         6, 17, 34, 19, 20,  0, 21, 20, 23, 24,  6, 17, 34, 19, 20,  0, 21, 20,
        23, 24,  6, 17, 34, 19, 20,  0, 21, 20, 23, 24,  6, 17, 34, 19, 20,  0,
        21, 20, 23, 24,  6, 17, 34, 19, 20,  0], device='cuda:0')

In [ ]:
charcount

148